# descSPIM-Python
This program is a control application for descSPIM in notebook format. By following the guide and launching Micro-Mnager, the program GUI, it is possible to control all devices at once.

By using this program, the following objectives can be achieved

・Automation of imaging

・Ensure reproducibility of imaging (reduction of manual errors)



#### 1. confirmation items
After completing the setup according to the installation guide, run the cells in order to confirm the installation contents. 2.

#### 2. control program
After confirming 1. above, run the cells according to the guide to enable imaging according to your purpose.

# 2. Control Program

#### Install Thorlabs related dll files

In [1]:
import clr
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\ThorLabs.MotionControl.KCube.StepperMotorCLI.dll")
clr.AddReference("C:\Program Files\Thorlabs\Kinesis\ThorLabs.MotionControl.KCube.DCServoCLI.dll")
clr.AddReference("C:\Program Files\Thorlabs\Kinesis\Thorlabs.MotionControl.KCube.StepperMotorCLI.dll")

from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.GenericMotorCLI.ControlParameters import *
from Thorlabs.MotionControl.GenericMotorCLI.AdvancedMotor import *
from Thorlabs.MotionControl.GenericMotorCLI.Settings import *
from Thorlabs.MotionControl.KCube.DCServoCLI import *
from Thorlabs.MotionControl.KCube.StepperMotorCLI import *

DeviceManagerCLI.BuildDeviceList()
devices = DeviceManagerCLI.GetDeviceList()
print(devices)


System.Collections.Generic.List`1[System.String]


#### Other Installations

In [2]:
import time
import os
import sys
import hid # Library for USB Devices
import math
import datetime
import clr #pythonnet
from ctypes import *
from pymmcore_plus import CMMCorePlus, DeviceType
from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QWidget
from pymmcore_widgets import StageWidget
from pymmcore_plus import CMMCorePlus

#### Loading batch file and launching Micro-Manger

In Micro-Manager, select the batch file in the same path as the cell below and proceed to the next step.

In [3]:
import subprocess

batch_file = r"C:/Users/DBSB/Desktop/231221_batch_camera_YN.bat"

subprocess.Popen(batch_file, shell=True)

<Popen: returncode: None args: 'C:/Users/DBSB/Desktop/231221_batch_camera_YN...>

If the above code does not get you up and running, try the following (Otherwise, skip it.)

In [4]:
# !"C:/Users/DBSB/Downloads/231220_batch_34555.bat"

#### Initialize the Thorlabs Kinesis library

The following cells should be executed only once and then not re-run.

In [4]:
import sys
import os
from ctypes import cdll

if sys.version_info < (3, 8):
    os.chdir(r"C:\Program Files\Thorlabs\Kinesis")
else:
    os.add_dll_directory(r"C:\Program Files\Thorlabs\Kinesis")

lib = cdll.LoadLibrary("Thorlabs.MotionControl.KCube.DCServo.dll")

#### Homing of two stages

Allow 60 seconds for the motor to return to home (default 0 point); check the numerical display of the motor on the KDC101 to confirm that it has returned to 0.0000 mm. *If the motor does not return to home within 60 seconds, increase the maximum waiting time (in seconds) for completion of homing at any time.

In [5]:
from PyQt5.QtCore import QThread
from pymmcore_plus import CMMCorePlus
import time

class HomingThread(QThread):
    def __init__(self, mmc, max_wait_time):
        QThread.__init__(self)
        self.mmc = mmc
        self.max_wait_time = max_wait_time

    def run(self):
        try:
            self.mmc.home("sampleStage")
            self.mmc.home("cameraStage")

            start_time = time.time()
            while time.time() - start_time < self.max_wait_time:
                sample_position = self.mmc.getPosition("sampleStage")
                camera_position = self.mmc.getPosition("cameraStage")
                if sample_position == self.mmc.getPosition("sampleStage") and camera_position == self.mmc.getPosition("cameraStage"):
                    break
                time.sleep(0.1)

            print("Sample stage homed. Current position: ", self.mmc.getPosition("sampleStage"))
            print("Camera stage homed. Current position: ", self.mmc.getPosition("cameraStage"))
        except Exception as e:
            print("An error occurred:", e)

mmc = CMMCorePlus().instance()
mmc.loadSystemConfiguration("C:/Users/DBSB/Desktop/231221_stage_YN.cfg")
homing_thread = HomingThread(mmc, 60)　# Change the value here from 60 in some cases
homing_thread.start()

Sample stage homed. Current position:  0.0
Camera stage homed. Current position:  0.0


#### First, both motors are set to Vel: 1 mm/s, Acc: 2 mm/s^2 for alignment.

In [5]:
# See: https://optomechs.com/pdf/manuals/KDC101/KDC101_ManualforAPT.pdf

  CC_SetVelParams(serial_num, 396, 773000) is determined by the device unit and can be checked from the kinesis application if necessary.

In [6]:
from ctypes import c_char_p, c_int

# Assuming 'lib' is already defined and appropriate functions are loaded from the DLL

# Serial numbers for the two motors
serial_nums = [c_char_p(b"27260577"), c_char_p(b"27260136")]

# Iterate through each serial number to set the velocity
for serial_num in serial_nums:
    # Open the device
    if lib.TLI_BuildDeviceList() == 0:
        lib.CC_Open(serial_num)
        lib.CC_StartPolling(serial_num, c_int(200))

        # Set the velocity
        lib.CC_SetVelParams(serial_num, 396, 773000)

        # Close the device
        lib.CC_Close(serial_num)

print("Velocity set for both motors")


Velocity set for both motors


#### Alignment determination by arbitrary speed (for more detailed alignment)

In this cell, you can enter any speed in the text box in the GUI. This cell is used when you want to speed up or slow down the speed from the upper cell to determine a finer alignment.
You may use this cell from the beginning to freely determine the speed.

In [7]:
from tkinter import Tk, Label, Entry, Button, Frame
from ctypes import c_char_p, c_int

# Assuming 'lib' is already defined and appropriate functions are loaded from the DLL

def set_velocity(serial_num, velocity_entry):
    velocity_mm_s = float(velocity_entry.get())
    # Convert the velocity from mm/s to device units
    velocity_device_unit = int(velocity_mm_s * 773000)  # Conversion formula based on provided information
    
    # Open the device
    if lib.TLI_BuildDeviceList() == 0:
        lib.CC_Open(serial_num)
        lib.CC_StartPolling(serial_num, c_int(200))

        # Set the velocity
        lib.CC_SetVelParams(serial_num, 396, velocity_device_unit)

        # Close the device
        lib.CC_Close(serial_num)
    print(f"Velocity set for {serial_num.value.decode()}")

# GUI setup
root = Tk()
root.title("Velocity Setter")

# Serial numbers and their labels
motor_info = [(c_char_p(b"27260577"), "sampleStage"), (c_char_p(b"27260136"), "cameraStage")]

for serial_num, label in motor_info:
    frame = Frame(root)
    frame.pack()

    Label(frame, text=f"Enter Velocity for {label} ({serial_num.value.decode()}) (mm/s):").pack(side="left")
    velocity_entry = Entry(frame)
    velocity_entry.pack(side="left")
    
    Button(frame, text="Set Velocity", command=lambda s=serial_num, v=velocity_entry: set_velocity(s, v)).pack(side="right")

root.mainloop()


Velocity set for 27260577
Velocity set for 27260136


#### Start-up of motor controller (setting of starting position)

To find the focal point manually, execute the following cells. You can track the current position by checking the Poll checkbox in the lower right corner of the window.

In [7]:
from PyQt5.QtWidgets import QApplication, QWidget, QHBoxLayout, QGroupBox
from pymmcore_plus import CMMCorePlus, DeviceType
from pymmcore_widgets import StageWidget
import sys

class MainApplication(QWidget):
    def __init__(self, mmc):
        super().__init__()
        self.setLayout(QHBoxLayout())
        self.mmc = mmc
        self.initUI()

    def initUI(self):
        try:
            stage_dev_list = list(self.mmc.getLoadedDevicesOfType(DeviceType.Stage))

            for stage_dev in stage_dev_list:
                device_type = self.mmc.getDeviceType(stage_dev)
                if device_type is DeviceType.Stage:
                    bx = QGroupBox("Z Control")
                    bx.setLayout(QHBoxLayout())
                    bx.layout().addWidget(StageWidget(device=stage_dev))
                    self.layout().addWidget(bx)
                else:
                    print(f"Device {stage_dev} is not a Stage device, it's a {device_type}")
        except Exception as e:
            print(f"An error occurred: {e}")

def on_exit():
  
    print("Terminate application")

def main():
    app = QApplication([])
    app.aboutToQuit.connect(on_exit)  # Set up processing at the end of the day

    mmc = CMMCorePlus().instance()
    try:
        mmc.loadSystemConfiguration("C:/Users/DBSB/Desktop/231221_stage_YN.cfg")
        mainApp = MainApplication(mmc)
        mainApp.show()
        sys.exit(app.exec())  # Exit the application using sys.exit() before calling exec()
    except Exception as e:
        print("An error occurred:", e)

if __name__ == '__main__':
    main()


アプリケーションを終了します


SystemExit: 0

C:\Users\DBSB\myenv\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Acquisition of start position

The program acquires the initial position.

In [8]:
mmc = CMMCorePlus.instance()

sampleStart = mmc.getPosition("sampleStage")
cameraStart = mmc.getPosition("cameraStage")
print("Initial sampleStage position: ", sampleStart)
print("Initial cameraStage position: ", cameraStart)

Initial sampleStage position:  48.18405440601939
Initial cameraStage position:  56.31601794241065


#### Start-up of motor controller (setting of end position)

In [9]:
from pymmcore_plus import CMMCorePlus, DeviceType
from PyQt5.QtWidgets import QApplication, QWidget, QHBoxLayout, QGroupBox
from pymmcore_widgets import StageWidget

app = QApplication.instance()
if app is None:
    app = QApplication([])

mmc = CMMCorePlus().instance()

config_file = "C:/Users/DBSB/Desktop/231221_stage_YN.cfg"
mmc.loadSystemConfiguration(config_file)

wdg = QWidget()
wdg.setLayout(QHBoxLayout())

try:
    stage_dev_list = list(mmc.getLoadedDevicesOfType(DeviceType.Stage))

    for stage_dev in stage_dev_list:
        device_type = mmc.getDeviceType(stage_dev)
        if device_type is DeviceType.Stage:
            bx = QGroupBox("Z Control")
            bx.setLayout(QHBoxLayout())
            bx.layout().addWidget(StageWidget(device=stage_dev))
            wdg.layout().addWidget(bx)
        else:
            print(f"Device {stage_dev} is not a Stage device, it's a {device_type}")
except Exception as e:
    print(f"An error occurred: {e}")

wdg.show()

app.exec_()


アプリケーションを終了します


0

#### Acquisition of end position

The program obtains the initial position.

In [10]:
from pymmcore_plus import CMMCorePlus

mmc = CMMCorePlus.instance()

sampleEnd = mmc.getPosition("sampleStage")
cameraEnd = mmc.getPosition("cameraStage")
print("New sampleStage position: ", sampleEnd)
print("New cameraStage position: ", cameraEnd)

sampleDistance = sampleEnd - sampleStart
cameraDistance = cameraEnd - cameraStart
print("sampleStage Distance: ", sampleDistance)
print("cameraStage Distance: ", cameraDistance)

New sampleStage position:  94.13977716683549
New cameraStage position:  108.46476631457098
sampleStage Distance:  45.955722760816094
cameraStage Distance:  52.14874837216033


In [ ]:
# https://pypi.org/project/pyKDC101/
!pip install pyKDC101
!pip install thorlabs-apt
# "C:\Program Files\Thorlabs\APT\APT Server\APT.dll"から'C:\Users\DBSB\myenv\lib\site-packages\thorlabs_apt\APT.dll'にコピー

In [ ]:
# pip install matplotlib
# pip uninstall PyQt6

#### Calculate and set cameraSpeed from sampleSpeed and Distance

In [ ]:
from tkinter import Tk, Label, Entry, Button, Frame
from ctypes import c_char_p, c_int

def set_velocity(serial_num, velocity_mm_s):
    global sampleSpeed, cameraSpeed

    # Convert the velocity from mm/s to device units
    velocity_device_unit = int(velocity_mm_s * 773000)  # Conversion formula based on provided information
    
    # Open the device
    if lib.TLI_BuildDeviceList() == 0:
        lib.CC_Open(serial_num)
        lib.CC_StartPolling(serial_num, c_int(200))

        # Set the velocity
        lib.CC_SetVelParams(serial_num, 396, velocity_device_unit)

        # Store the velocity in the appropriate global variable
        if serial_num.value.decode() == "27260577":
            sampleSpeed = velocity_mm_s
            # Calculate cameraSpeed based on sampleSpeed
            cameraSpeed = sampleSpeed / (sampleDistance / cameraDistance)
            set_velocity(c_char_p(b"27260136"), cameraSpeed)  # Set velocity for the camera
        elif serial_num.value.decode() == "27260136":
            cameraSpeed = velocity_mm_s

        # Close the device
        lib.CC_Close(serial_num)
    print(f"Velocity set for {serial_num.value.decode()}")

# GUI setup
root = Tk()
root.title("Velocity Setter")

# Only Serial number for sampleStage
serial_num, label = c_char_p(b"27260577"), "sampleStage"
frame = Frame(root)
frame.pack()

Label(frame, text=f"Enter Velocity for {label} ({serial_num.value.decode()}) (mm/s):").pack(side="left")
velocity_entry = Entry(frame)
velocity_entry.pack(side="left")

Button(frame, text="Set Velocity", command=lambda: set_velocity(serial_num, float(velocity_entry.get()))).pack(side="right")

root.mainloop()


#### Check speed and distance traveled

In [ ]:
class MotorControlApp:
    def __init__(self, sampleSpeed, cameraSpeed, sampleStart, cameraStart, sampleEnd, cameraEnd):
        self.sampleSpeed = float(sampleSpeed)
        self.cameraSpeed = float(cameraSpeed)
        self.sampleStart = float(sampleStart)
        self.cameraStart = float(cameraStart)
        self.sampleEnd = float(sampleEnd)
        self.cameraEnd = float(cameraEnd)

motorControlApp = MotorControlApp(sampleSpeed, cameraSpeed, sampleStart, cameraStart, sampleEnd, cameraEnd)

sampleDistance = motorControlApp.sampleEnd - motorControlApp.sampleStart
cameraDistance = motorControlApp.cameraEnd - motorControlApp.cameraStart

if cameraDistance != 0:
    print("Sample speed = " + str(motorControlApp.sampleSpeed) + " mm/s")
    print("Camera speed = " + str(motorControlApp.cameraSpeed) + " mm/s")
    print("The sample will move " + str(sampleDistance) + " mm")
    print("The camera will move " + str(cameraDistance) + " mm")
else:
    print("Error: Camera distance is zero.")

#### Return the motor to the starting position.

In [ ]:
print("Available devices:", mmc.getLoadedDevices())


In [13]:
mmc.loadSystemConfiguration("C:/Users/DBSB/Desktop/231221_stage_YN.cfg")

mmc.setPosition("sampleStage", sampleStart)

mmc.setPosition("cameraStage", cameraStart)

print("sampleStage is returning to sampleStart: ", mmc.getPosition("sampleStage"))
print("cameraStage is returning to cameraStart: ", mmc.getPosition("cameraStage"))

sampleStage returned to:  94.13977716683549
cameraStage returned to:  108.46476631457098


#### Calculate the number of shots

Execute the following cell to confirm the number of shots. After confirming the number of shots, launch Multi-D Acq. and check the Time Points box in the upper left corner of the window, enter the calculated number of shots in the count box, and set interval to 0 mm/s. From the Image settings in Micro-Manger Home, set Exposure to 200 ms. Set Exposure to 200 ms.

In [14]:
images = sampleDistance/(0.24*sampleSpeed)
print(int(images))

3829


#### Execution of Shooting

With the following cell running, select Aquire on the Multi-D Acq. of the Micro-Manager; the selection is transmitted to the program via Arduino and the following cell is executed.

In [ ]:
import time
import serial
import threading
from pymmcore_plus import CMMCorePlus

# Initialize Serial Connection and CMMCorePlus
COM = "COM6"
bitRate = 19200
ser = serial.Serial(COM, bitRate, timeout=1)
mmc = CMMCorePlus.instance()

config_file = "C:/Users/DBSB/Desktop/231221_stage_YN.cfg"
mmc.loadSystemConfiguration(config_file)

devices = mmc.getLoadedDevices()
print("Loaded devices:", devices)

if "sampleStage" in devices and "cameraStage" in devices:
    print("Both stages are loaded correctly.")
else:
    print("Error: Stages not loaded. Check device configuration.")

def move_stage(stage_label, absolute_position):
    try:
        mmc.setPosition(stage_label, absolute_position)
        mmc.waitForDevice(stage_label)
        final_position = mmc.getPosition(stage_label)
        print(f"Moved {stage_label} to {final_position}")
    except Exception as e:
        print(f"Error moving {stage_label}: {e}")

def move_stages_simultaneously(sampleEnd, cameraEnd):
    
    threads = [
        threading.Thread(target=move_stage, args=("sampleStage", sampleEnd)),
        threading.Thread(target=move_stage, args=("cameraStage", cameraEnd))
    ]

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

def read_serial_and_move_stages():
    while True:
        data = ser.read(512)
        pickup = data[0:1]
        if pickup.decode("utf-8") == "1":
            print("Triggering stage movement...")
            move_stages_simultaneously(sampleEnd, cameraEnd)
        elif pickup.decode("utf-8") == "0":
            print("Standby for movement")
        time.sleep(0.1)

serial_thread = threading.Thread(target=read_serial_and_move_stages)
serial_thread.daemon = True
serial_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Program terminated")


Press ■ from the top control bar of jupyter notebook to finish executing the above code. Specify the destination of the captured image from Micro-Manager and save it.